In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.io import imread

import tensorflow as tf

from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, GlobalAveragePooling2D, Lambda, Reshape, multiply
from keras.layers import Activation, Dropout, BatchNormalization, AveragePooling2D
from keras.optimizers import Adam
from keras.applications import ResNet50
from keras.utils import to_categorical
from keras import backend as K
from keras.layers import Layer
from sklearn.model_selection import train_test_split

from keras.optimizers import RMSprop,SGD
from keras.regularizers import l2

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
'''physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)'''

"physical_devices = tf.config.experimental.list_physical_devices('GPU')\nif len(physical_devices) > 0:\n    tf.config.experimental.set_memory_growth(physical_devices[0], True)"

In [4]:
#!unzip /content/AML.zip -d /content/

In [5]:
#!unzip /content/CALL.zip -d /content/

In [6]:
#!pip install imgaug

In [2]:
import os
from PIL import Image
import numpy as np
import imgaug.augmenters as iaa

# Ruta de la carpeta de descargas
download_dir = os.path.expanduser("AML")  # Ruta de la carpeta de descargas

# Subdirectorios dentro de la carpeta de descargas
subdirs = ["all", "M1", "M2", "M3", "M4", "M5", "M6", "M7", "sanos"]

# Carpeta para guardar imágenes aumentadas
output_dir_base = os.path.join(download_dir, "AML1")
os.makedirs(output_dir_base, exist_ok=True)

# Configuración de aumentadores
seq_gaussian_noise = iaa.Sequential([iaa.AdditiveGaussianNoise(scale=(0, 0.2*255))])
seq_brightness = iaa.Sequential([iaa.Multiply((1.2, 1.5))])
seq_rotation = iaa.Sequential([iaa.Affine(rotate=25)])
seq_grayscale = iaa.Sequential([iaa.Grayscale(alpha=(0.0, 1.0))])
seq_horizontal_flip = iaa.Sequential([iaa.Fliplr(1.0)])
seq_vertical_flip = iaa.Sequential([iaa.Flipud(1.0)])
seq_shear = iaa.Sequential([iaa.Affine(shear=(-25, 25))])
seq_zoom = iaa.Sequential([iaa.Affine(scale=(1.5, 1.5))])

# Listas para almacenar imágenes y recuentos
directories = []
dircount = []

print("Leyendo imágenes de", download_dir)

# Recorrer cada subdirectorio
for subdir in subdirs:
    subdir_path = os.path.join(download_dir, subdir)
    output_dir = os.path.join(output_dir_base, subdir)
    os.makedirs(output_dir, exist_ok=True)  # Create subdirectory if it doesn't exist
    images_count = 0

    # Recorrer archivos en el subdirectorio
    for root, _, filenames in os.walk(subdir_path):
        for filename in filenames:
            if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                filepath = os.path.join(root, filename)

                # Leer la imagen y convertirla a array de numpy
                image = np.array(Image.open(filepath))

                # Guardar la imagen original en el mismo subdirectorio
                output_path_original = os.path.join(output_dir, filename[:-4] + "_original.jpg")
                Image.fromarray(image).convert("RGB").save(output_path_original)

                # Aplicar ruido gaussiano
                image_gaussian = seq_gaussian_noise.augment_image(image)
                output_path_gaussian = os.path.join(output_dir, filename[:-4] + "_g.jpg")
                Image.fromarray(image_gaussian).convert("RGB").save(output_path_gaussian)

                # Aplicar aumento de brillo
                image_brightness = seq_brightness.augment_image(image)
                output_path_brightness = os.path.join(output_dir, filename[:-4] + "_b.jpg")
                Image.fromarray(image_brightness).convert("RGB").save(output_path_brightness)

                # Aplicar rotación a la izquierda de 90 grados
                image_rotation = seq_rotation.augment_image(image)
                output_path_rotation = os.path.join(output_dir, filename[:-4] + "_r.jpg")
                Image.fromarray(image_rotation).convert("RGB").save(output_path_rotation)

                # Convertir la imagen a formato RGB antes de aplicar grayscaling
                image_rgb = Image.fromarray(image).convert("RGB")
                image_grayscale = seq_grayscale.augment_image(np.array(image_rgb))

                # Guardar la imagen grayscaled en el mismo subdirectorio
                output_path_grayscale = os.path.join(output_dir, filename[:-4] + "_gray.jpg")
                Image.fromarray(image_grayscale).save(output_path_grayscale)

                # Aplicar reflexión horizontal
                image_horizontal_flip = seq_horizontal_flip.augment_image(image)
                output_path_hflip = os.path.join(output_dir, filename[:-4] + "_hflip.jpg")
                Image.fromarray(image_horizontal_flip).convert("RGB").save(output_path_hflip)

                # Aplicar reflexión vertical
                image_vertical_flip = seq_vertical_flip.augment_image(image)
                output_path_vflip = os.path.join(output_dir, filename[:-4] + "_vflip.jpg")
                Image.fromarray(image_vertical_flip).convert("RGB").save(output_path_vflip)

                # Aplicar shearing
                image_shear = seq_shear.augment_image(image)
                output_path_shear = os.path.join(output_dir, filename[:-4] + "_shear.jpg")
                Image.fromarray(image_shear).convert("RGB").save(output_path_shear)

                # Aplicar zoom con un factor de 0.2
                image_zoom = seq_zoom.augment_image(image)
                output_path_zoom = os.path.join(output_dir, filename[:-4] + "_zoom.jpg")
                Image.fromarray(image_zoom).convert("RGB").save(output_path_zoom)

                images_count += 1

    # Almacenar recuento y directorio
    dircount.append(images_count)
    directories.append(subdir)

# Mostrar estadísticas finales
print('Directorios leídos:', len(directories))
print("Imágenes en cada directorio:", dircount)
print('Suma total de imágenes en subdirectorios:', sum(dircount))

Leyendo imágenes de AML
Directorios leídos: 9
Imágenes en cada directorio: [40, 40, 40, 40, 40, 40, 40, 40, 40]
Suma total de imágenes en subdirectorios: 360


In [8]:
# Ruta de la carpeta de descargas
# Directorio de descargas
download_dir = os.path.expanduser("AML/AML1")  # Ruta de la carpeta de descargas

# Subdirectorios dentro de la carpeta de descargas
#subdirs = ["M0", "M1", "M2", "M3", "M4"]
subdirs = ["all", "M1", "M2", "M3", "M4", "M5", "M6", "M7", "sanos"]

# Listas para almacenar imágenes y recuentos
images = []
directories = []
dircount = []

# Tamaño deseado de las imágenes redimensionadas
target_height = 150 #108, 200
target_width = 250  #160, 300

print("leyendo imagenes de ",download_dir)

# Recorrer cada subdirectorio
for subdir in subdirs:
    subdir_path = os.path.join(download_dir, subdir)
    images_count = 0

    # Recorrer archivos en el subdirectorio
    for root, _, filenames in os.walk(subdir_path):
        for filename in filenames:
            if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                filepath = os.path.join(root, filename)

                # Leer y redimensionar la imagen
                image = imread(filepath)
                image_resized = resize(image, (target_height, target_width), anti_aliasing=True, preserve_range=True)
                images.append(image_resized)
                #images.append(image)

                images_count += 1

    # Almacenar recuento y directorio
    dircount.append(images_count)
    directories.append(subdir)

# Mostrar estadísticas finales
print('Directorios leídos:', len(directories))
print("Imágenes en cada directorio:", dircount)
print('Suma total de imágenes en subdirectorios:', sum(dircount))

# Convertir la lista de imágenes a un array de numpy
#images_array = np.array(images)
'''for i in range(len(images)):
  images[i] = np.array(images[i], dtype=np.uint8)'''
# Convertir las imágenes a RGB
'''for i in range(len(images)):
  images[i] = images[i][..., :3]
'''
type(images)

leyendo imagenes de  AML/AML1


Directorios leídos: 7
Imágenes en cada directorio: [360, 360, 360, 360, 360, 360, 360]
Suma total de imágenes en subdirectorios: 2520


list

In [9]:
# Directorio que contiene las imágenes
directorio_imagenes = os.path.expanduser("AML/AML1")
directorio_imagenes

'AML/AML1'

# Nueva sección

---



In [10]:
labels = []
indice = 0
for cantidad in dircount:
  for i in range(cantidad):
    labels.append(indice)
  indice = indice + 1
print("Cantidad de etiquetas creadas: ", len(labels))

Cantidad de etiquetas creadas:  2520


In [11]:
AMLI=[]
indice=0
for directorio in directories:
    name = directorio.split(os.sep)
    print(indice , name[len(name)-1])
    AMLI.append(name[len(name)-1])
    indice=indice+1

0 M1
1 M2
2 M3
3 M4
4 M5
5 M6
6 M7


In [12]:
type(AMLI)

list

In [13]:
AMLI

['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7']

In [14]:
#for img in images:
    #print(img.shape)

In [15]:
y = np.array(labels)
#X = images
X = np.array(images, dtype=np.uint8) #convierto de lista a numpy

# Find the unique numbers from the train labels
classes = np.unique(y)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)
type(X)

Total number of outputs :  7
Output classes :  [0 1 2 3 4 5 6]


numpy.ndarray

In [16]:
'''print(y[0])
plt.imshow(X[0])
plt.show()
print(y[1])
plt.imshow(X[1])
plt.show()
print(y[2])
plt.imshow(X[2])
plt.show()
print(y[3])
plt.imshow(X[3])
plt.show()
print(y[4])
plt.imshow(X[4])
plt.show()
print(y[5])
plt.imshow(X[5])
plt.show()
print(y[6])
plt.imshow(X[6])
plt.show()
print(y[7])
plt.imshow(X[7])
plt.show()'''

'print(y[0])\nplt.imshow(X[0])\nplt.show()\nprint(y[1])\nplt.imshow(X[1])\nplt.show()\nprint(y[2])\nplt.imshow(X[2])\nplt.show()\nprint(y[3])\nplt.imshow(X[3])\nplt.show()\nprint(y[4])\nplt.imshow(X[4])\nplt.show()\nprint(y[5])\nplt.imshow(X[5])\nplt.show()\nprint(y[6])\nplt.imshow(X[6])\nplt.show()\nprint(y[7])\nplt.imshow(X[7])\nplt.show()'

In [17]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir listas a arrays NumPy
#X_train = np.array(X_train)
#X_test = np.array(X_test)

# Normalizamos como float32 para poder dividirlos entre 0 y 1
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Valores entre 0..1
X_train /= 255
X_test /= 255

print('Training data shape : ', X_train.shape, y_train.shape)
print('Testing data shape : ', X_test.shape, y_test.shape)

img_size = X[0].shape
print(img_size)

Training data shape :  (2016, 150, 250, 3) (2016,)
Testing data shape :  (504, 150, 250, 3) (504,)
(150, 250, 3)


In [18]:
from keras.utils import to_categorical

# Codificar las etiquetas
y_train_encoded = to_categorical(y_train, num_classes=nClasses)
y_test_encoded = to_categorical(y_test, num_classes=nClasses)

# Modelos


---

